# QR decomposition

Alberto Quaini

### Import some libraries

In [1]:
import numpy as np
from scipy import linalg as la

## Problem 1

In [32]:
def mod_gram_schmidt(A):
    """ Compute the modified Gram-Schmidt QR decomposition of A
    Input: A, array(m, n)
    Output: Q, array(m, n) orthonormal
            R, array(n, n) upper triangular
    """
    
    m, n = A.shape
    r = np.linalg.matrix_rank(A)
    if r < n:
        print("A is not full column rank")
    Q = np.copy(A)
    R = np.zeros((n, n))
    
    for i in range(n):
        R[i,i] = la.norm(Q[:,i])
        Q[:,i] = Q[:,i] / R[i,i]
        for j in range(i+1, n):
            R[i,j] = np.dot(np.transpose(Q[:,j]), Q[:,i])
            Q[:,j] = Q[:,j] - R[i,j] * Q[:,i]
    
    return Q, R

In [34]:
A = np.random.random((6,4))
Q, R = mod_gram_schmidt(A)

In [24]:
print("Is R upper triangular? ", np.allclose(np.triu(R), R))

Is R upper triangular?  True


In [25]:
print("Is Q orthonormal? ", np.allclose(Q.T @ Q, np.identity(4)))

Is Q orthonormal?  True


In [26]:
print("Is true that A = QR? ", np.allclose(Q @ R, A))

Is true that A = QR?  True


In [27]:
Q1, R1 = la.qr(A, mode="economic")

In [28]:
Q

array([[ 0.2248691 ,  0.43841357, -0.57187107,  0.17488342],
       [ 0.57714052, -0.07993732, -0.22608113, -0.75951318],
       [ 0.28004844,  0.15622821,  0.76516383, -0.10031163],
       [ 0.39698409,  0.35205515,  0.02119293,  0.4284213 ],
       [ 0.6121215 , -0.46635656,  0.03664516,  0.43301822],
       [ 0.07500986,  0.65997351,  0.18596531, -0.10697436]])

In [29]:
Q1

array([[-0.2248691 ,  0.43841357,  0.57187107, -0.17488342],
       [-0.57714052, -0.07993732,  0.22608113,  0.75951318],
       [-0.28004844,  0.15622821, -0.76516383,  0.10031163],
       [-0.39698409,  0.35205515, -0.02119293, -0.4284213 ],
       [-0.6121215 , -0.46635656, -0.03664516, -0.43301822],
       [-0.07500986,  0.65997351, -0.18596531,  0.10697436]])

In [30]:
R

array([[ 1.39264544,  1.18523377,  0.7250062 ,  0.56935842],
       [ 0.        ,  1.33108372,  0.8113073 ,  0.55801341],
       [ 0.        ,  0.        ,  0.82587151, -0.28696591],
       [ 0.        ,  0.        ,  0.        ,  0.25104842]])

In [31]:
R1

array([[-1.39264544, -1.18523377, -0.7250062 , -0.56935842],
       [ 0.        ,  1.33108372,  0.8113073 ,  0.55801341],
       [ 0.        ,  0.        , -0.82587151,  0.28696591],
       [ 0.        ,  0.        ,  0.        , -0.25104842]])

## Problem 2

In [57]:
def det_QR(A):
    """ Compute the determinant of invertible matrix A
    via QR decomposition
    """
    
    m, n = A.shape
    r = np.linalg.matrix_rank(A)
    if not n == m:
        print("A is not square")
    if r < n:
        print("A is not invertible")
    
    Q, R = la.qr(A, mode="economic")
    return abs(np.prod(np.diag(R)))

In [58]:
A = np.random.random((6,6))

In [59]:
r = round(det_QR(A), 4)
print("Rank of A is: ", r)

Rank of A is:  0.0438


In [60]:
r1 = round(abs(la.det(A)), 4)
print("Rank of A via scipy linear algebra package is: ", r1)

Rank of A via scipy linear algebra package is:  0.0438


Function in one line:

In [61]:
det_oneline = lambda A: abs(np.prod(np.diag(la.qr(A, mode='economic')[1])))

In [62]:
# Test
r2 = round(det_oneline(A), 4)
print("Rank of A from one-line function is: ", r2)

Rank of A from one-line function is:  0.0438


## Problem 3

In [153]:
def lin_sys(A, b):
    
    m, n = A.shape
    l = len(b)
    r = np.linalg.matrix_rank(A)
    if not n == m:
        print("A is not square")
    if r < n:
        print("A is not invertible")
    if not l == n:
        print("b is not of the right length")
        
    Q, R = la.qr(A, mode = 'economic')
    y = np.dot(np.transpose(Q), b)
    x = np.zeros(n)

    for j in range(n-1, -1, -1):
        x[j] = (y[j] - np.dot(R[j, j+1:], x[j+1:])) / R[j,j]
    
    return x

In [154]:
A = np.random.random((4, 4))
b = np.random.random(4)
x = lin_sys(A, b)
print(x)

[ 0.5284044   0.6562337  -0.41446825 -0.03076377]


In [155]:
np.linalg.solve(A, b)

array([ 0.5284044 ,  0.6562337 , -0.41446825, -0.03076377])

## Problem 4

In [102]:
sign = lambda x: 1 if x >= 0 else -1

In [119]:
def householder(A):
    """ Compute QR decomposition of A via householder method
    """
    
    m, n = A.shape
    r = np.linalg.matrix_rank(A)
    if r < n:
        print("A is not full column rank")
    
    R = np.copy(A)
    Q = np.eye(m)
    
    for k in range(n):
        u = np.copy(R[k:,k])
        u[0] = u[0] + sign(u[0]) * la.norm(u)
        u = u / la.norm(u)
        R[k:,k:] = R[k:,k:] - 2 * np.outer(u, np.dot(np.transpose(u), R[k:,k:]))
        Q[k:,:] = Q[k:,:] - 2 * np.outer(u, np.dot(np.transpose(u), Q[k:,:]))
    
    return np.transpose(Q), R

In [120]:
A = np.random.random((5, 3))
Q,R = householder(A)    

In [121]:
print("Is R upper triangular? ", np.allclose(np.triu(R), R))

Is R upper triangular?  True


In [122]:
print("Is Q orthonormal? ", np.allclose(Q.T @ Q, np.identity(5)))

Is Q orthonormal?  True


In [123]:
print("Is true that A = QR? ", np.allclose(Q @ R, A))

Is true that A = QR?  True


## Problem 5

In [145]:
def hessenberg(A):
    """ Compute Hessenberg QR decomposition of A
    """
    
    m, n = A.shape
    r = np.linalg.matrix_rank(A)
    if not n == m:
        print("A is not square")
    if r < n:
        print("A is not invertible")
        
    H = np.copy(A)
    Q = np.eye(m)
    
    for k in range(n-2):
        u = np.copy(H[k+1:,k])
        u[0] = u[0] + sign(u[0]) * la.norm(u)
        u = u / la.norm(u)
        H[k+1:,k:] = H[k+1:,k:] - 2 * np.outer(u, np.dot(np.transpose(u), H[k+1:,k:]))
        H[:,k+1:] = H[:,k+1:] - 2 * np.outer(np.dot(H[:,k+1:], u), np.transpose(u))
        Q[k+1:,:] = Q[k+1:,:] - 2 * np.outer(u, np.dot(np.transpose(u), Q[k+1:,:]))
        
    return H, np.transpose(Q)

In [146]:
A = np.random.random((8,8))
H, Q = hessenberg(A)

In [147]:
np.allclose(np.triu(H, -1), H)

True

In [144]:
np.allclose(Q @ H @ Q.T, A)

True